In [ ]:
%%spark
--conf spark.yarn.queue=root.zw02.hadoop-wallepnc.query
--conf spark.driver.maxResultSize=4g
--conf spark.driver.memory=8g
--conf spark.executor.memory=8g
--conf spark.executor.cores=2
--conf spark.yarn.executor.memoryOverhead=2072
--conf spark.dynamicAllocation.enabled=true
--conf spark.dynamicAllocation.minExecutors=600
--conf spark.dynamicAllocation.maxExecutors=1000
--conf spark.sql.shuffle.partitions=6000
--conf spark.memory.fraction=0.7
--conf spark.sql.autoBroadcastJoinThreshold=1048576000

,Driver日志,队列信息
,/home/sankuai/logs/spark-1611822319.log,root.zw02.hadoop-wallepnc.query


CmdOutput(['tail', '-50', '/home/sankuai/logs/spark-1611822319.log'], 2)

In [ ]:
%%sql t --preview --quiet
SELECT get_json_object(`debug`,'$.planningData.obstacleDecider.decisions') as obstacles,
       autocar_name,
       timestamp,
       frame_id,
       `git-branch`,
       main_decision,
       hour,
       record_name,
       hdmap_version,
       heading,
       get_json_object(`debug`,'$.planningData.intention.intentionType') as intention_type,
       get_json_object(`debug`,'$.planningData.intention.refL') as ref_l,
       is_auto
  FROM mart_wallepnc_hulk.detail_planning
 where  dt='20210114'
   and autocar_name = 'zcx-04'
    and frame_id % 10 = 0
   and get_json_object(`debug`,'$.planningData.obstacleDecider.decisions') is not null
   and get_json_object(`debug`,'$.planningData.obstacleDecider.decisions.type') not like '%NO_NUDGE%'

In [ ]:

db = 'mart_wallepnc_hulk'

In [ ]:
%%sql r --preview --quiet

create table if not exists {db}.bdecision_environment
(
    obstacles string comment 'obstacle information',
    autocar_name string COMMENT '主车的名字,例：mkz-00',
    `timestamp` double COMMENT '时间戳,UNIXtime,单位为s',
    `frame_id` bigint COMMENT 'frame sequence',
    `git-branch` string COMMENT 'git branch name',
     `main_decision` string COMMENT '自车决策：cruise、stop、estop、change_lane、mission_complete、not_ready、parking',
    `hour` string COMMENT '两位数24小时制表示小时',
    record_name string COMMENT "record_name",
    `hdmap_version` string COMMENT '高精地图的名字+版本',
     `heading` double COMMENT 'heading, 正东方向是0,朝北是正值',
    intention_type string comment 'intention',
    ref_l double comment 'ref_l',
     `is_auto` string COMMENT '是否处于自动驾驶模式,字符串true或false'
) comment 'bdecision_environment Demo表' stored as orc

In [ ]:
t.createOrReplaceTempView('my_tmp_view')

In [ ]:
%%sql r --preview --quiet

insert into {db}.bdecision_environment
select * from my_tmp_view

In [ ]:
%%sql r --preview --quiet
select * from {db}.bdecision_environment